In [4]:
text = "This is the text I would like to chunk up. It is the example text for this exercise"

In [7]:
chunk_size = 35
chunks=[]

In [8]:


for i in range(0, len(text), chunk_size):
    chunk=(text[i:i+chunk_size])
    chunks.append(chunk)
print(chunks)

['This is the text I would like to ch', 'unk up. It is the example text for ', 'this exercise']


In [9]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)

In [11]:
text_splitter.create_documents([text])

[Document(page_content='This is the text I would like to ch'),
 Document(page_content='unk up. It is the example text for '),
 Document(page_content='this exercise')]

In [12]:
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=4, separator='')
text_splitter.create_documents([text])

[Document(page_content='This is the text I would like to ch'),
 Document(page_content='o chunk up. It is the example text'),
 Document(page_content='ext for this exercise')]

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text = """
One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear.

Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor's, you don't get half as many customers. You get no customers, and you go out of business.

It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer. [1]
"""

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 65, chunk_overlap=0)

In [16]:
text_splitter.create_documents([text])

[Document(page_content="One of the most important things I didn't understand about the"),
 Document(page_content='world when I was a child is the degree to which the returns for'),
 Document(page_content='performance are superlinear.'),
 Document(page_content='Teachers and coaches implicitly told us the returns were linear.'),
 Document(page_content='"You get out," I heard a thousand times, "what you put in." They'),
 Document(page_content='meant well, but this is rarely true. If your product is only'),
 Document(page_content="half as good as your competitor's, you don't get half as many"),
 Document(page_content='customers. You get no customers, and you go out of business.'),
 Document(page_content="It's obviously true that the returns for performance are"),
 Document(page_content='superlinear in business. Some think this is a flaw of'),
 Document(page_content='capitalism, and that if we changed the rules it would stop being'),
 Document(page_content='true. But superlinear returns for

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 450, chunk_overlap=0)
text_splitter.create_documents([text])

[Document(page_content="One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear."),
 Document(page_content='Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor\'s, you don\'t get half as many customers. You get no customers, and you go out of business.'),
 Document(page_content="It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer. [1]")]

### Level 3: Document Specific Splitting (json,html,markdown,python, java)

### Level 4: Semantic Chunking- The hypothesis is that semantically similar chunks should be held together.

In [23]:
with open('essay.txt') as file:
    essay = file.read()

In [24]:
import re

# Splitting the essay on '.', '?', and '!'
single_sentences_list = re.split(r'(?<=[.?!])\s+', essay)
print (f"{len(single_sentences_list)} senteneces were found")

385 senteneces were found


In [25]:
sentences = [{'sentence': x, 'index' : i} for i, x in enumerate(single_sentences_list)]
sentences[:3]

[{'sentence': "\t\n\nA Student's Guide to Startups\n\n Want to start a startup?",
  'index': 0},
 {'sentence': 'Get funded by Y Combinator.', 'index': 1},
 {'sentence': 'October 2006\n\n(This essay is derived from a talk at MIT.)\n\nTill recently graduating seniors had two choices: get a job or go to grad school.',
  'index': 2}]

Great, now that we have our sentences, I want to combine the sentence before and after so that we reduce noise and capture more of the relationships between sequential sentences.

Let's create a function so we can use it again. The buffer_size is configurable so you can select how big of a window you want. Keep this number in mind for the later steps. I'll just use buffer_size=1 for now.

In [26]:
def combine_sentences(sentences, buffer_size=1):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

sentences = combine_sentences(sentences)

In [28]:
sentences[:3]

[{'sentence': "\t\n\nA Student's Guide to Startups\n\n Want to start a startup?",
  'index': 0,
  'combined_sentence': "\t\n\nA Student's Guide to Startups\n\n Want to start a startup? Get funded by Y Combinator."},
 {'sentence': 'Get funded by Y Combinator.',
  'index': 1,
  'combined_sentence': "\t\n\nA Student's Guide to Startups\n\n Want to start a startup? Get funded by Y Combinator. October 2006\n\n(This essay is derived from a talk at MIT.)\n\nTill recently graduating seniors had two choices: get a job or go to grad school."},
 {'sentence': 'October 2006\n\n(This essay is derived from a talk at MIT.)\n\nTill recently graduating seniors had two choices: get a job or go to grad school.',
  'index': 2,
  'combined_sentence': 'Get funded by Y Combinator. October 2006\n\n(This essay is derived from a talk at MIT.)\n\nTill recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup.'}]

Sometime finish the above code  https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [29]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings())

In [31]:
with open("essay.txt") as f:
    essays = f.read()

In [32]:
docs = text_splitter.create_documents([essays])
print(docs[0].page_content)

	

A Student's Guide to Startups

 Want to start a startup? Get funded by Y Combinator. October 2006

(This essay is derived from a talk at MIT.)

Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be? I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups. I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's. The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college? In fact, why go to college at all? Why not start a startup instead? A year and a half ago I gave a

In [33]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile"
)

In [35]:
docs = text_splitter.create_documents([essays])
print(docs[0].page_content)

	

A Student's Guide to Startups

 Want to start a startup? Get funded by Y Combinator. October 2006

(This essay is derived from a talk at MIT.)

Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be? I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups. I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's. The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college? In fact, why go to college at all? Why not start a startup instead? A year and a half ago I gave a

In [36]:
print(len(docs))

21


In [37]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="standard_deviation"
)

docs = text_splitter.create_documents([essays])
print(docs[0].page_content)

	

A Student's Guide to Startups

 Want to start a startup? Get funded by Y Combinator. October 2006

(This essay is derived from a talk at MIT.)

Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be? I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups. I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's. The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college? In fact, why go to college at all? Why not start a startup instead? A year and a half ago I gave a

In [38]:
print(len(docs))

3


In [40]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)

docs = text_splitter.create_documents([essays])
print(docs[0].page_content)

	

A Student's Guide to Startups

 Want to start a startup? Get funded by Y Combinator. October 2006

(This essay is derived from a talk at MIT.)

Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be? I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups. I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's. The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college? In fact, why go to college at all? Why not start a startup instead? A year and a half ago I gave a

In [41]:
print(len(docs))

16


Level 5: Agentic Chunking